<a href="https://colab.research.google.com/github/naokityokoyama/Deep_Learing/blob/main/Deep_learning_Carros_usados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U scikit-learn==0.23.0

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
df = pd.read_csv('/content/1.1 autos.csv', encoding='latin1')

In [ ]:
df.head(2)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50


In [3]:
df = df.drop(['dateCrawled', 'dateCreated', 'nrOfPictures','postalCode', 'lastSeen'], axis=1)

In [4]:
df['name'].value_counts()

Ford_Fiesta                                                       657
BMW_318i                                                          627
Opel_Corsa                                                        622
Volkswagen_Golf_1.4                                               603
BMW_316i                                                          523
                                                                 ... 
VW_Polo_6N_1_6_75PS_mit_Klimaanlage                                 1
MAZDA_MX5_!!!_850_FESTPREIS_!!!                                    1
SMART_FORTWO_CABRIO_PASSION_KLIMA_GUTER_ZUSTAND_VIELE_NEUTEILE      1
Trabant_zum_Wiederaufbau__oder_fuer_Ersatzteile                     1
Triumph_TR_7_Cabrio_H_Kennzeichen                                   1
Name: name, Length: 233531, dtype: int64

In [5]:
df = df.drop('name', axis=1)

In [ ]:
df['seller'].value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [6]:
df = df.drop('seller', axis=1)

In [7]:
df = df.drop('offerType', axis=1)

In [ ]:
#preco abaixo de 10
abaixo10 = df.loc[df.price<=10]

In [8]:
df = df[df.price>10]

In [ ]:
#acima de 350000

acima350 = df.loc[df.price>350000]

In [9]:
df = df[df.price < 350000]

In [10]:
#NaN veicleType
df.loc[pd.isnull(df['vehicleType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [11]:
df['vehicleType'].value_counts() #limousine

limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64

In [ ]:
df.loc[pd.isnull(df['gearbox'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371443,3300,control,kombi,2006,NaN,0,touran,150000,7,diesel,volkswagen,NaN
371460,3500,control,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN
371486,350,control,kleinwagen,1996,NaN,65,punto,150000,0,NaN,fiat,NaN
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN


In [ ]:
df['gearbox'].value_counts() #manuell

manuell      266547
automatik     75508
Name: gearbox, dtype: int64

In [ ]:
df.loc[pd.isnull(df['model'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371399,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,fiat,ja
371476,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,sonstige_autos,ja
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein


In [ ]:
df['model'].value_counts() #golf

golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: model, Length: 251, dtype: int64

In [ ]:
df.loc[pd.isnull(df['fuelType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371496,3850,test,cabrio,2006,manuell,108,2_reihe,125000,2,NaN,peugeot,nein
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [ ]:
df['fuelType'].value_counts() #benzin 

benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64

In [ ]:
df.loc[pd.isnull(df['notRepairedDamage'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371507,5999,test,kombi,2005,manuell,140,a4,150000,4,diesel,audi,NaN
371514,999,control,cabrio,2000,manuell,95,megane,150000,4,benzin,renault,NaN
371515,1690,test,kombi,2004,manuell,55,fabia,150000,4,benzin,skoda,NaN
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN


In [ ]:
df['notRepairedDamage'].value_counts() #nein 

nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64

In [12]:
valores = {'vehicleType' : 'limousine', 'gearbox': 'manuell',  
           'model' : 'golf' ,'fuelType': 'benzin', 'notRepairedDamage' :'nein' }

In [ ]:
valores

{'fuelType': 'benzin',
 'gearbox': 'manuell',
 'model': 'golf',
 'notRepairedDamage': 'nein',
 'vehicleType': 'limousine'}

In [13]:
df = df.fillna(value=valores)

In [14]:
df.isnull().sum()

price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

In [ ]:
df.head(2)

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja


In [15]:
X = df.iloc[:, 1:13].values
y = df.iloc[:,0].values

In [16]:
le = LabelEncoder()

In [17]:
lista_le = [0,1,3,5,8,9,10]

In [18]:
for i in lista_le:
  X[:, i] = le.fit_transform(X[:,i])

In [19]:
X[0:20]

array([[1, 6, 1993, 1, 0, 118, 150000, 0, 1, 38, 1],
       [1, 3, 2011, 1, 190, 118, 125000, 5, 3, 1, 0],
       [1, 7, 2004, 0, 163, 119, 125000, 8, 3, 14, 1],
       [1, 4, 2001, 1, 75, 118, 150000, 6, 1, 38, 1],
       [1, 4, 2008, 1, 69, 103, 90000, 7, 3, 31, 1],
       [1, 6, 1995, 1, 102, 11, 150000, 10, 1, 2, 0],
       [1, 2, 2004, 1, 109, 8, 150000, 8, 1, 25, 1],
       [0, 1, 2014, 1, 125, 61, 30000, 8, 1, 10, 1],
       [1, 4, 1998, 1, 101, 118, 150000, 0, 1, 38, 1],
       [0, 6, 2004, 1, 105, 10, 150000, 12, 1, 19, 1],
       [0, 5, 2005, 1, 140, 171, 150000, 12, 3, 38, 0],
       [0, 5, 1995, 1, 115, 171, 150000, 11, 1, 38, 1],
       [0, 5, 2004, 1, 131, 171, 150000, 2, 1, 38, 1],
       [0, 7, 2011, 1, 190, 161, 70000, 3, 3, 23, 1],
       [1, 4, 1910, 1, 0, 130, 5000, 0, 1, 10, 1],
       [1, 6, 2016, 1, 60, 174, 150000, 0, 1, 38, 1],
       [0, 4, 2004, 0, 75, 228, 150000, 2, 1, 27, 1],
       [1, 1, 2007, 1, 136, 61, 150000, 6, 3, 10, 1],
       [1, 1, 2004, 1, 102,

In [20]:
#one hot encoder
onehotencoder = ColumnTransformer(transformers=[('OneHot', 
                                                 OneHotEncoder(), lista_le)], remainder='passthrough')
X = onehotencoder.fit_transform(X).toarray()

In [ ]:
#rede neural 

#316 entradas + 1 saida

regressor = Sequential()
regressor.add(Dense(units=158, activation='relu', input_dim=316))
regressor.add(Dense(units=158, activation='relu'))
regressor.add(Dense(units=1, activation='linear'))

regressor.compile(loss='mean_absolute_error', optimizer='adam',    #absolute_erro ->abs
                  metrics = ['mean_absolute_error'])  

regressor.fit(X,y, batch_size=300, epochs=100)  

In [ ]:
y_pred = regressor.predict(X)

In [ ]:
y_pred.mean() , y.mean()

(5395.7866, 5916.833945186492)

Funcao para rede neural

In [ ]:
def criarRede():
  
  regressor = Sequential()
  regressor.add(Dense(units=158, activation='relu', input_dim=316))
  regressor.add(Dense(units=158, activation='relu'))
  regressor.add(Dense(units=1, activation='linear'))

  regressor.compile(loss='mean_absolute_error', optimizer='adam',    #absolute_erro ->abs
                  metrics = ['mean_absolute_error'])  
  return regressor

Cross Validation

In [ ]:
regressor = KerasRegressor(build_fn=criarRede, epochs=100, batch_size=300)

Cross = cross_val_score(estimator=regressor , X=X, y=y, cv=10, scoring="neg_mean_absolute_error")

Epoch 1/100
1078/1078 [==============================] - 7s 6ms/step - loss: 4287.8629 - mean_absolute_error: 4287.8629
Epoch 2/100
1078/1078 [==============================] - 6s 5ms/step - loss: 3403.3527 - mean_absolute_error: 3403.3527
Epoch 3/100
1078/1078 [==============================] - 6s 5ms/step - loss: 3092.3902 - mean_absolute_error: 3092.3902
Epoch 4/100
1078/1078 [==============================] - 6s 5ms/step - loss: 2915.8273 - mean_absolute_error: 2915.8273
Epoch 5/100
1078/1078 [==============================] - 6s 5ms/step - loss: 2840.7057 - mean_absolute_error: 2840.7057
Epoch 6/100
1078/1078 [==============================] - 6s 5ms/step - loss: 2795.1551 - mean_absolute_error: 2795.1551
Epoch 7/100
1078/1078 [==============================] - 6s 5ms/step - loss: 2734.1181 - mean_absolute_error: 2734.1181
Epoch 8/100
1078/1078 [==============================] - 6s 5ms/step - loss: 2696.6548 - mean_absolute_error: 2696.6548
Epoch 9/100
1078/1078 [=================